In [ ]:
from google.colab import files
uploaded = files.upload()

Saving academy_awards.csv to academy_awards.csv


## Quick Review
- `requests.request("GET", url, params=querystring)`

- `response.json()`

In [ ]:
import requests

# Simple interactive script querying an API

url = "http://www.omdbapi.com/"

title = input("Enter a movie title that you like: ")

# Populate the query string with your OMDb apikey and title
querystring = {"apikey":"5cf98de", "t":title}

# Use the request method
# Returns a requests.models.Request obj
response = requests.request("GET", url, params=querystring)
json_as_dict = response.json()
print(type(json_as_dict))

# Iterate over all key/value pairs in the dictionary and print them
for key, value in json_as_dict.items():
    print(key, ' : ', value)

Enter a movie title that you like: Atlantis
<class 'dict'>
Title  :  Atlantis
Year  :  2013–2015
Rated  :  N/A
Released  :  23 Nov 2013
Runtime  :  45 min
Genre  :  Adventure, Drama, Fantasy, Romance
Director  :  N/A
Writer  :  Johnny Capps, Julian Murphy, Howard Overman
Actors  :  Mark Addy, Jack Donnelly, Robert Emms, Aiysha Hart
Plot  :  Far from home, Jason washes up on the shores of the ancient and mysterious city of Atlantis.
Language  :  English
Country  :  UK
Awards  :  4 nominations.
Poster  :  https://m.media-amazon.com/images/M/MV5BN2Y1YzAyZjgtMDI0NS00MjM0LWExNWQtYjE2NDAyOGExNzIxXkEyXkFqcGdeQXVyOTQxNzM2MjY@._V1_SX300.jpg
Ratings  :  [{'Source': 'Internet Movie Database', 'Value': '6.6/10'}]
Metascore  :  N/A
imdbRating  :  6.6
imdbVotes  :  10,396
imdbID  :  tt2705602
Type  :  series
totalSeasons  :  2
Response  :  True


## EXERCISE:  
Before we display the _key/value_ pairs in the `dict`, let's first check that we get proper data back from the API. 

- Check the OMDb API in the Firefox Browser to determine WHAT is returned from
 - A movie title that IS in their database.
 - A movie title that is NOT in their database.
- What is the **key** to look at?  What is its **value**?
- Incorporate an `if/else` block that tells the user when a movie was NOT found in the Open Movie Database.

In [ ]:
import requests

# Add some basic error checking to the prior example

url = "http://www.omdbapi.com/"

title = input("Enter a movie that you like: ")

# Populate the query string with your apikey and title
querystring = {"apikey":"9b96d509", "t":title}

response = requests.request("GET", url, params=querystring)

json_as_dict = response.json()

# Why do we compare against a string?
if json_as_dict["Response"] == "False":
    print(f"Could not find a movie in the OMDb API database.")
    print(json_as_dict["Error"])
else:
    for key, value in json_as_dict.items():
        print(key, ' : ', value)

## Let's do something a bit different.

In [ ]:
import requests
import webbrowser

# Get JSON data from an API and use one of the fields.

url = "http://www.omdbapi.com/"
apikey = "5cf98de"

title = input("Enter a movie that you like and we'll show you the Movie Theater poster when it was released: ")

# Populate the query string with your apikey and title
querystring = {"apikey":apikey, "t":title}
response = requests.request("GET", url, params=querystring)

# Use the json method in requests module to convert the json to a dict
json_as_dict = response.json()
poster_key = "Poster"

# Get the value associated with the key of interest
poster_value = json_as_dict[poster_key]
print(poster_value)
# Open the URL associated with Poster
# webbrowser.open_new(poster_value)

Enter a movie that you like and we'll show you the Movie Theater poster when it was released: 42
https://m.media-amazon.com/images/M/MV5BMTQwMDU4MDI3MV5BMl5BanBnXkFtZTcwMjU1NDgyOQ@@._V1_SX300.jpg


## Use TWO Data Sources to Solve a Problem

**INPUTS**: 
- Request that the user enter any movie title.
- Read in a CSV data file that has information on Academy Award Nominations and Winners through 2010

*OUTPUT*: Report on how many Academy Awards that the movie was NOMINATED for AND whether or not the movie won an **Academy Award for Best Picture**.

**How we'll solve this:**
- Read in the `academy_awards.csv` file which has all Academy Awards information.
- Make an API call to the Open Movie Database to see if the OMDb API states that it won SOME award.  If not, the `Awards` key will have value `N/A`.
- Look through the CSV datafile to see if that movie was NOMINATED for an Academy Award (and how many).
- Determine whether or not the movie won **Best Picture** for that year.

## Let's breakdown the code in parts.

### Read in the CSV data file.

In [ ]:
import requests
import csv

# Use a GET request to fetch API data and utilize a second data source to get additional information

url = "http://www.omdbapi.com/"
apikey = "5cf98de"

# This is for program design and testing.
# You can 'hardcode' movies that we're sure won Best Picture and did NOT win Best Picture.
# title = "Gone with the Wind"
# title = "Wizard of Oz"

academy_awards = []
awards_file = 'academy_awards.csv'

# Open the CSV with Best Picture information and read into Python dict
with open(awards_file, encoding='utf-8', errors='ignore') as csv_file:
    csv_reader = csv.DictReader(csv_file, dialect='unix')
    academy_awards.extend(csv_reader)

print(f"There are {len(academy_awards)} rows in the {awards_file} dataset.")

There are 10137 rows in the academy_awards.csv dataset.


### Make the API call to Open Movie Database (OMDb).

In [ ]:
title = input("Enter a movie title and we'll check if it won the Academy Award for Best Picture: ")

# Populate the query string with your apikey and title
querystring = {"apikey":apikey, "t":title}
movie_requested = requests.request("GET", url, params=querystring)

# Use the json method in requests module to convert the json to a dict
movie_requested_json = movie_requested.json()


Enter a movie title and we'll check if it won the Academy Award for Best Picture: inception


### Use the `Awards` field in OMDb to determine if we want to query our the Academy Award movie from the CSV to see if the movie won any **Academy Awards**.

In [ ]:
# Some error-checking.  See if we get response data.
if (movie_requested_json['Response'] == "False"):
    print(f"Your requested movie {title} is not available in the OMDb Database.")

else:
    movie_requested_title = movie_requested_json["Title"]
    movie_requested_awards = movie_requested_json["Awards"]
    movie_requested_year = movie_requested_json["Year"]

    # First check if the movie won ANY awards as per the API.
    if movie_requested_awards == "N/A":
            print(f"{movie_requested_title} did not win any awards according to OMDb.")
            # exit()
    else:  # Build a list of dictionaries that includes the award that this movie was nominated for
        print(f"\nAccording to OMDb, {movie_requested_title} DID win some awards in {movie_requested_year}.\n")
        print("So let's check if it won any ACADEMY AWARDS ...\n")
        award_list = []
        for m in academy_awards:
            if (movie_requested_title.lower() == m["Nominee"].lower()):
                award_list.append(m)

        if len(award_list) > 0:
            print(f"According to {awards_file}, this movie was nominated for {len(award_list)} ACADEMY AWARDS in {movie_requested_year}.")
            print("\n ... Now let's check if it won the Academy Award for Best Picture ... \n")

            best_picture = False
            for a in award_list:
                if (("Best Picture" in a["Category"]) and ("YES" in a["Won?"]) ):
                    print(f"And {a['Nominee']} DID win the Academy Award for Best Picture in {movie_requested_year}.")
                    best_picture = True
                    break
            if best_picture == False:
                print(f"Nope, {movie_requested_title} DID NOT win the Academy Award for Best Picture in {movie_requested_year}.\n")
        else:
            print(f"According to {awards_file}, this movie did NOT WIN ANY ACADEMY AWARDS in {movie_requested_year}.\n")



According to OMDb, Inception DID win some awards in 2010.

So let's check if it won any ACADEMY AWARDS ...

According to academy_awards.csv, this movie was nominated for 8 ACADEMY AWARDS in 2010.

 ... Now let's check if it won the Academy Award for Best Picture ... 

Nope, Inception DID NOT win the Academy Award for Best Picture in 2010.



### The full code in one cell

In [ ]:
import requests
import csv

# Use a GET request to fetch API data and utilize a second data source to get additional information

url = "http://www.omdbapi.com/"
apikey = "5cf98de"

# This is for program design and testing.
# We 'hardcode' movies that we're sure won Best Picture and did NOT win Best Picture.
# title = "Gone with the Wind"
# title = "Wizard of Oz"

academy_awards = []
awards_file = 'academy_awards.csv'

# Open the CSV with Best Picture information and read into Python dict
with open(awards_file, encoding='utf-8', errors='ignore') as csv_file:
    csv_reader = csv.DictReader(csv_file, dialect='unix')
    academy_awards.extend(csv_reader)

title = input("Enter a movie title and we'll check if it won the Academy Award for Best Picture: ")

# Populate the query string with your apikey and title
querystring = {"apikey":apikey, "t":title}
movie_requested = requests.request("GET", url, params=querystring)

# Use the json method in requests module to convert the json to a dict
movie_requested_json = movie_requested.json()

# Some error-checking.  See if we get response data.
if (movie_requested_json['Response'] == "False"):
    print(f"Your requested movie {title} is not available in the OMDb Database.")

else:
    movie_requested_title = movie_requested_json["Title"]
    movie_requested_awards = movie_requested_json["Awards"]
    movie_requested_year = movie_requested_json["Year"]

    # First check if the movie won ANY awards as per the API.
    if movie_requested_awards == "N/A":
            print(f"{movie_requested_title} did not win any awards according to OMDb.")
            # exit()
    else:  # Build a list of dictionaries that includes the award that this movie was nominated for
        print(f"\nAccording to OMDb, {movie_requested_title} DID win some awards in {movie_requested_year}.\n")
        print("So let's check if it won any ACADEMY AWARDS ...\n")
        award_list = []
        for m in academy_awards:
            if (movie_requested_title.lower() == m["Nominee"].lower()):
                award_list.append(m)

        if len(award_list) > 0:
            print(f"According to {awards_file}, this movie was nominated for {len(award_list)} ACADEMY AWARDS in {movie_requested_year}.")
            print(" ... Now let's check if it won the Academy Award for Best Picture ... \n")

            best_picture = False
            for a in award_list:
                if (("Best Picture" in a["Category"]) and ("YES" in a["Won?"]) ):
                    print(f"And {a['Nominee']} DID win the Academy Award for Best Picture in {movie_requested_year}.")
                    best_picture = True
                    break
            if best_picture == False:
                print(f"Nope, {movie_requested_title} DID NOT win the Academy Award for Best Picture in {movie_requested_year}.\n")
        else:
            print(f"According to {awards_file}, this movie did NOT WIN ANY ACADEMY AWARDS in {movie_requested_year}.\n")


Enter a movie title and we'll check if it won the Academy Award for Best Picture: inception

According to OMDb, Inception DID win some awards in 2010.

So let's check if it won any ACADEMY AWARDS ...

According to academy_awards.csv, this movie was nominated for 8 ACADEMY AWARDS in 2010.
 ... Now let's check if it won the Academy Award for Best Picture ... 

Nope, Inception DID NOT win the Academy Award for Best Picture in 2010.



# Workshop:  More Public API Examples

# Exercise: Modify the code above to display the list of ALL Academy Awards won, not only BEST PICTURE.

- Copy the above code in the cell below.
- Questions to ask before you modify anything
 - At what point do I know, for sure, that the movie was NOMINATED for an Academy Award?
 - What value determines that the movie DID win SOME Academy Award?
 - What value determines the SPECIFIC Academy Award that was won?
 - What did we do when we found that the movie won BEST PICTURE?  What modifications do you need to make to the code to make sure we check for ALL awards won?


In [ ]:
# Modify your Academy Award Code here.

import requests
import csv

# Use a GET request to fetch API data and utilize a second data source to get additional information

url = "http://www.omdbapi.com/"
apikey = "5cf98de"

# This is for program design and testing.
# We 'hardcode' movies that we're sure won Best Picture and did NOT win Best Picture.
# title = "Gone with the Wind"
# title = "Wizard of Oz"

academy_awards = []
awards_file = 'academy_awards.csv'

# Open the CSV with Best Picture information and read into Python dict
with open(awards_file, encoding='utf-8', errors='ignore') as csv_file:
    csv_reader = csv.DictReader(csv_file, dialect='unix')
    academy_awards.extend(csv_reader)

title = input("Enter a movie title and we'll check if it won an Academy Award: ")

# Populate the query string with your apikey and title
querystring = {"apikey":apikey, "t":title}
movie_requested = requests.request("GET", url, params=querystring)

# Use the json method in requests module to convert the json to a dict
movie_requested_json = movie_requested.json()

# Some error-checking.  See if we get response data.
if (movie_requested_json['Response'] == "False"):
    print(f"Your requested movie {title} is not available in the OMDb Database.")

else:
    movie_requested_title = movie_requested_json["Title"]
    movie_requested_awards = movie_requested_json["Awards"]
    movie_requested_year = movie_requested_json["Year"]

    # First check if the movie won ANY awards as per the API.
    if movie_requested_awards == "N/A":
            print(f"{movie_requested_title} did not win any awards according to OMDb.")
            # exit()
    else:  # Build a list of dictionaries that includes the award that this movie was nominated for
        print(f"\nAccording to OMDb, {movie_requested_title} DID win some awards in {movie_requested_year}.\n")
        print("So let's check if it was nominated for any ACADEMY AWARDS ...\n")
        award_list = []
        for m in academy_awards:
            if (movie_requested_title.lower() == m["Nominee"].lower()):
                award_list.append(m)

        if len(award_list) > 0:
            print(f"This movie was nominated for {len(award_list)} ACADEMY AWARDS in {movie_requested_year}.")
            print(" ... Now let's check if it won an Academy Award ... \n")

#(("Best Picture" in a["Category"]))

            awardWin = False
            for a in award_list:
                if ("YES" in a["Won?"]) :
                    print(f"{a['Nominee']} won the Academy Award for {a['Category']} in {movie_requested_year}.")
                    awardWin = True
                    #break
            if awardWin == False:
                print(f"Nope, {movie_requested_title} DID NOT win the Academy Award for Best Picture in {movie_requested_year}.\n")
        else:
            print(f"According to {awards_file}, this movie did NOT WIN ANY ACADEMY AWARDS in {movie_requested_year}.\n")



Enter a movie title and we'll check if it won an Academy Award: inception

According to OMDb, Inception DID win some awards in 2010.

So let's check if it was nominated for any ACADEMY AWARDS ...

This movie was nominated for 8 ACADEMY AWARDS in 2010.
 ... Now let's check if it won an Academy Award ... 

Inception won the Academy Award for Cinematography in 2010.
Inception won the Academy Award for Sound in 2010.
Inception won the Academy Award for Sound Editing in 2010.
Inception won the Academy Award for Visual Effects in 2010.


## EXERCISE:  Another Example of Writing API Data to a CSV

Again, we'll use the Open Movie Database.

This time we'll read movies from a **text file** (it is neither CSV nor JSON data).
For each movie in the file, we'll make a call to OMDb
If the movie exists in the database, we'll write the returned data to a CSV file.
- Upload `movie_list.txt`.  Feel free to edit this and add a few movies.
- Use `open()` and `readlines()` to read the `movie_list.txt` data into a list.
- Create an empty list called `filtered_movies_list` 
- Walk through the list of movie titles that you read in using `for`
 - Assemble the *query string* that you'll pass to `request`.
 - Use `requests.request()` to make your API call to OMDb
 - Use the earlier example for error-checking the response
   - If the movie title was found, then add it to `filtered_movies_list`, otherwise tell the user that *No such movie was found*.
- Look at last night's solution for how to write the `filtered_movies_list` to a CSV file
 - HINT:  Use this for the fieldnames that you'll pass to `csv.DictWriter`
 
 `fieldnames = filtered_movies_list[0].keys()`

In [ ]:
import requests
import json
import csv

# Make multiple API calls and create a CSV file from the data that you used.

url = "http://www.omdbapi.com/"

# Use this key if you need one for the OMDb API
myapikey = "5cf98de"
querystring = {"apikey": myapikey}

# Read the text file
textFile = open("movie_list.txt", "r")
lines = textFile.readlines()
textFile.close()

#Create an empty list called filtered_movies_list
filtered_movies_list = []
successfulAPI = []

counter = 0
for n in lines:
  if n.endswith('\n'):
    lines[counter] = n[:-1]
    counter += 1
  else:
    counter +=1

print(lines)


#Walk through the list of movie titles that you read in using for
#Assemble the query string that you'll pass to request.
#Use requests.request() to make your API call to OMDb
#Use the earlier example for error-checking the response
#If the movie title was found, then add it to filtered_movies_list, otherwise tell the user that No such movie was found.

for n in lines:
  querystring = {"apikey":myapikey, "t":n}
  movie_requested = requests.request("GET", url, params=querystring)
  movie_requested_json = movie_requested.json()
  if (movie_requested_json['Response'] == "False"):
    print(f"{n} is not available in the OMDb Database.")
  else:
    filtered_movies_list.append(n)
    successfulAPI.append(movie_requested_json)


# Create a new list where you'll store the successful API calls
print(filtered_movies_list)
print(successfulAPI)

# Traverse the list of movies that you read in 
# and make a call to the OMDb Movie API.

# If it's a successful API call, add it to your new list

# Write the new list to a CSV file
print()
print("------------------------------------------------")
print()

print("Creating the CSV file...")

fields = ["Title", "Year", "Rated", "Released", "Runtime", "Genre", "Director", "Writer", "Actors", "Plot", "Language", "Country", "Awards", "Poster", "Ratings", "Metascore", "imdbRating", "imdbVotes", "imdbID", "Type", "DVD", "BoxOffice", "Production", "Website", "Response"]
with open('filteredList.csv', mode='w') as csvfile:
  csvwriter = csv.DictWriter(csvfile, fieldnames=fields)
  csvwriter.writeheader()
  for s in successfulAPI:
      csvwriter.writerow(s)

print("Complete!")


['The Godfather', 'Gone With the Wind', 'Casablanca', 'This is not a movie', 'Jaws', 'Field of Dreams', 'Moneyball', 'The Natural', '42', 'Driving Miss Daisy', 'Grumpy Old Men', 'The Notebook', 'The Shawshank Redemption', 'The Green Mile', 'Pinocchio', 'Foo Bar Baz Bar Baz', 'Rear Window', 'Big Trouble in Little China', 'The Ninth Gate', '2001: A Space Odyssey', '12 Angry Men', 'National Treasure', 'Raiders of the Lost Ark', 'Gone Girl', 'Saving Private Ryan', '1917', 'Dunkirk', 'The Avengers: Endgame', 'Kingsman: the secret service', 'Casino Royale', 'Skyfall', 'Robot Monster']
Foo Bar Baz Bar Baz is not available in the OMDb Database.
The Avengers: Endgame is not available in the OMDb Database.
['The Godfather', 'Gone With the Wind', 'Casablanca', 'This is not a movie', 'Jaws', 'Field of Dreams', 'Moneyball', 'The Natural', '42', 'Driving Miss Daisy', 'Grumpy Old Men', 'The Notebook', 'The Shawshank Redemption', 'The Green Mile', 'Pinocchio', 'Rear Window', 'Big Trouble in Little Chi

## Exercise:  Extend the above code to also create a JSON file with the following `keys`:
- `"movies"`: which contains the list of movie titles.
- `"highest_metascore"` : which contains the movie `Title` with the highest "Metascore"
- `"longest_duration"` : which contains the movie `Title` with the longest runtime.

In [ ]:
#Create a dictionary
json_data = {}

print(successfulAPI)

#Create and populate keys in this dictionary
json_data['movies'] = filtered_movies_list
json_data['highest_metascore'] = 0
json_data['longest_duration'] = 0

#------------------------------------------------------------------

#What's the highest metascore movie in the dataset?

metascores = []

#take scores
for n in successfulAPI:
  if n['Metascore'].isnumeric():
    metaNum = n['Metascore']
    #make into an int so we can sort
    metaNum = int(metaNum)
    metascores.append(metaNum)

#sort
metascores.sort(reverse=True)

#turn it back into a string
winner = str(metascores[0])

#pull the movie of that length
for n in successfulAPI:
  if n['Metascore'] == (winner):
    json_data['highest_metascore'] = n['Title']

#------------------------------------------------------------------

#What's the longest duration movie in the dataset?

movieTimes = []

#take durations
for n in successfulAPI:
  durationNum = n['Runtime']
  #get rid of "min"
  durationNum = durationNum[:-4]
  #make into an int so we can sort
  durationNum = int(durationNum)
  movieTimes.append(durationNum)

#sort
movieTimes.sort(reverse=True)

#turn it back into a string
winner = (str(movieTimes[0]) + " min")

#pull the movie of that length
for n in successfulAPI:
  if n['Runtime'] == (winner):
    json_data['longest_duration'] = n['Title']

#------------------------------------------------------------------

#Use json.dump() to write this dictionary to movie_list_winners.json.
with open("movie_list_winners.json", "w") as newdata:
    json.dump(json_data, newdata, indent=3, sort_keys=True)

print("complete!")



[{'Title': 'The Godfather', 'Year': '1972', 'Rated': 'R', 'Released': '24 Mar 1972', 'Runtime': '175 min', 'Genre': 'Crime, Drama', 'Director': 'Francis Ford Coppola', 'Writer': 'Mario Puzo (screenplay by), Francis Ford Coppola (screenplay by), Mario Puzo (based on the novel by)', 'Actors': 'Marlon Brando, Al Pacino, James Caan, Richard S. Castellano', 'Plot': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.', 'Language': 'English, Italian, Latin', 'Country': 'USA', 'Awards': 'Won 3 Oscars. Another 26 wins & 30 nominations.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.2/10'}, {'Source': 'Rotten Tomatoes', 'Value': '98%'}, {'Source': 'Metacritic', 'Value': '100/100'}], 'Metascore': '100', 'imdbRating': '9.2', 'imdbVotes': '1,573,189', 'imdbID': 'tt0068646', 'T